In [25]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2
import numpy as np
import datetime
from sktime_neuro.utils.pickle_wrapper import deserialise_if_exists, serialise_if_doesnt_exist
from sktime_neuro.datasets.matching_pennies_util import *
from sktime_neuro.transformations.multivariate_detrender import ColumnDetrender
from sktime_neuro.transformations.series_to_panel.eeg_epoching import epoch
from sktime_neuro.utils.mne_processing import create_annotation
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def detrend(data, n_jobs=1):
    dt = ColumnDetrender(data)
    return dt.detrend(n_jobs=n_jobs)

In [5]:
def process(sid):
    raw = get_raw(sid)
    annot = create_annotation(raw)
    data = raw.get_data().transpose()
    detrended = detrend(data, -1)
    sf = raw.info['sfreq']
    return (annot, detrended, sf)

def leaveOneOut(leaveOut: int):
    leftout = None
    datasets = []
    for i in range(5,12):
        if i == leaveOut:
            leftout = process(i)
        else:
            datasets.append(process(i))
    return datasets, leftout

In [6]:
datasets, leftout = leaveOneOut(5)

/home/patchouli/programming/foss/sktime-neuro/sktime_neuro/datasets/matching_pennies_util.py:10: RuntimeWarning: Did not find any eeg.json associated with sub-05_task-matchingpennies.

The search_str was "/home/patchouli/programming/foss/sktime-neuro/cache/eeg_matchingpennies/sub-05/**/eeg/sub-05*eeg.json"
  return read_raw_bids(bids_path=bidspath, verbose=False)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   21.7s remaining:   50.5s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   21.9s remaining:   21.9s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   26.5s remaining:   11.4s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.3s finished
/home/patchouli/programming/foss/sktime-neuro/sktime_neuro/datasets/matching_pennies_util.py:10: RuntimeWarning: Did not find any eeg.json associated with sub-06_task-matchingpennies.

The search_str was "/home/patchouli/programming/foss/sktim

In [7]:
onsetlabels = ["raised-right/match-true", "raised-right/match-false",
               "raised-left/match-true", "raised-left/match-false"]

def epoch_dataset(dataset):
    epochs, labels = epoch(dataset[1], dataset[0], onsetlabels, [-0.5, 0.5], sfreq=dataset[2])
    return (epochs, labels)

def epoch_all(datasets):
    epoched = []
    for dataset in datasets:
        epoched.append(epoch_dataset(dataset))

    return epoched

In [8]:
epoched = epoch_all(datasets)
l1a_epochs, l1a_labels = epoch_dataset(leftout)

In [27]:
rocket = RocketClassifier(n_jobs=-1)
metrics = np.empty([len(datasets),3])
i = 0
startime = datetime.datetime.now()
for trainset, trainlabs in epoched:
    fittime = datetime.datetime.now()
    rocket.fit(trainset, trainlabs)
    print(f"Fitting took {datetime.datetime.now() - fittime}")
    predtime = datetime.datetime.now()
    ypred = rocket.predict(l1a_epochs)
    print(f"Prediction took {datetime.datetime.now() - predtime}")
    #print(classification_report(ypred, l1a_labels))
    acc = accuracy_score(ypred, l1a_labels)
    f1 = f1_score(ypred, l1a_labels, average='weighted')
    precision = precision_score(ypred, l1a_labels, average='weighted')
    x = np.array([acc, f1, precision])
    metrics[i] = x
    i += 1
endtime = datetime.datetime.now()
print(f"Total test took {endtime - startime}")

Fitting took 0:01:32.840222
Prediction took 0:01:31.116539
[0.24666667 0.31106529 0.60818182]
[[2.46666667e-01 3.11065292e-01 6.08181818e-01]
 [9.72305369e-05 4.94269039e-05 1.01680893e-05]
 [2.31211362e-05 9.29802295e-06 1.64276672e-05]
 [9.83305369e-05 4.95268951e-05 1.06680935e-05]
 [2.83211377e-05 1.02980194e-05 1.49276741e-05]
 [9.96305369e-05 4.94268864e-05 1.04680976e-05]]


KeyboardInterrupt: 